In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from importlib import reload
import emcee
import numpy as np
import matplotlib.pyplot as plt
from spectra import Spectrum

In [2]:
from numpy import exp, sqrt, pi
def delta_l_from_delta_v(delta_velocity, lambda_rest):
    return delta_velocity*lambda_rest/299792.458 #d_v is in km/s not in m/s

def lnL(theta, x, y, yerr, lambda_rest, c):
    amp,sigma,delta_velocity, c = theta
    delta_lambda = delta_l_from_delta_v(delta_velocity,lambda_rest)
    model = c + (amp / (sqrt(2*pi) * sigma)) * exp(-(x-delta_lambda-lambda_rest)**2 / (2*sigma**2))
    inv_sigma2 = 1.0/(yerr**2)
    return -0.5*(np.sum((y-model)**2*inv_sigma2))

def lnprior(theta):
    amp,sigma,delta_velocity,c = theta
    if 0 < sigma < 10 and -100 < delta_velocity < 100:
        return 0.0
    return -np.inf

def lnprob(theta, x, y, yerr, lambda_rest, c):
    lp = lnprior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + lnL(theta, x, y, yerr, lambda_rest, c)

In [3]:
# problem3
from __future__ import print_function

import numpy as np
import emcee
import sys
import matplotlib.pyplot as plt
import corner


def run_emcee(x, y_obs, sigma, lambda_rest, c, ml_result): #ml_result is the result object from lmfit

    # Set up the properties of the problem.
    ndim, nwalkers = 4, 200

    # Setup a bunch of starting positions.
    pos = [ml_result + 1e-4*np.random.randn(ndim) for i in range(nwalkers)]
        
    # Create the sampler.
    sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob, args=(x, y_obs, sigma, lambda_rest, c))

    sampler.run_mcmc(pos, 500)
    samples = sampler.get_chain(discard=300, flat=True)
    return sampler, samples



def show_walkers(chain, labels, savefile=None):

    nwalkers, nreps, ndim = chain.shape

    xval = np.arange(0, nreps)

    fig, ax = plt.subplots(ncols=1, nrows=ndim, figsize=(14, 4))
    for i_dim in range(ndim):
        ax[i_dim].set_ylabel(labels[i_dim])
        
        for i in range(100):
            ax[i_dim].plot(xval, chain[i, :, i_dim], color='black', alpha=0.5)

    if savefile is not None:
        plt.savefig(savefile)

    return fig, ax


def show_corner_plot(samples,labels,truths, savefile=None):

    fig = corner.corner(samples, labels=labels, truths=truths, quantiles=[0.16,0.5,0.84], show_titles=True)

    
    if savefile is not None:
        plt.savefig(savefile)

    return fig

In [15]:
def fit_mcmc(filename, lambda_rest_arr, showPlots=False):
    spec = Spectrum(filename)
    velocities = []
    sigmas = []
    for lambda_rest in lambda_rest_arr:
        data = spec.fit_unique(lambda_rest)
        amp = data['result'].params['amp'].value
        sigma = data['result'].params['sigma'].value
        lambdaObs = data['result'].params['lambda0'].value
        delta_velocity = 299792.458*(lambdaObs-lambda_rest)/lambda_rest
        y0 = data['y'][0]
        p_initial = [amp,sigma,delta_velocity, y0]#values from result object
        #print(p_initial)
        sampler_gaussian, samples_gaussian = run_emcee(data['x'], data['y'], data['sigma'], lambda_rest, y0, p_initial)
        if showPlots:
            print("Walker and Corner plots for wavelentgh: "+str(lambda_rest))
            fig, axes = show_walkers(sampler_gaussian.chain, ['amp', 'sigma', 'velocity', 'c'])
            fig = show_corner_plot(samples_gaussian, ['amp','sigma','velocity','c'], p_initial)
        v_med=np.quantile(samples_gaussian[:,2],.50)
        dv = 0.5*(np.quantile(samples_gaussian[2,:],.84)-np.quantile(samples_gaussian[2,:],.16))
        ps = np.quantile(samples_gaussian[:,2],[.16,.50,.84])
        dv=(ps[2]-ps[0])/2
        velocities.append(v_med)
        sigmas.append(dv)
       
    print(velocities, sigmas)  
        
    
    
#spec = Spectrum('src/data-spectrum/new/idid000007077jd2458163p6789f000.fits')
#spec = Spectrum("src/daniel's-spectrum/idid000007172jd2458163p6789f000.fits")
#spec = Spectrum("src/data-spectrum/third_batch/idid000003632jd2458163p6789f000.fits")
#lambda_rest = 4861.297761
#spec.plot('-r',4340,4780)
#spec.gfitting([lT],False)

fit_mcmc("src/data-spectrum/new/idid000004031jd2458163p6789f000.fits",[4861.297761,6562.85175], True)



[65.15251168105675, 41.461486706071966] [8.460301198164789, 2.8604327933938833]
